In [1]:
import json
import random

random.seed(42)
# 读取原始JSON文件
data = []

with open('../data/angular_filtered/subsets/generation/test_race.json', 'r', encoding='utf-8') as f:
    race_data = json.load(f)
# with open('../data/angular_filtered/subsets/generation/test_gpt35_zeroshot.json', 'r', encoding='utf-8') as f:
#     zeroshot_data = json.load(f)
# with open('../data/angular_filtered/subsets/generation/test_gpt35_rag.json', 'r', encoding='utf-8') as f:
#     rag_data = json.load(f)
with open('../data/angular_filtered/subsets/generation/test_gpt35_model_classified_rag.json', 'r', encoding='utf-8') as f:
    model_rag_data = json.load(f)
# with open('../data/angular_filtered/subsets/generation/test_nngen.json', 'r', encoding='utf-8') as f:
#     nngen_data = json.load(f)
    

In [2]:
for index in range(len(model_rag_data)):
    item = {
        'diff': race_data[index]['diff'],
        # 'human': race_data[index]['msg'],
        'race': race_data[index]['race'],
        # 'zeroshot': zeroshot_data[index]['chatgpt_zeroshot'],
        # 'rag': rag_data[index]['chatgpt_rag'],
        'classified_rag': model_rag_data[index]['chatgpt_rag'],
        # 'nngen': nngen_data[index]['nngen'],
        'type': race_data[index]['type']
    }
    data.append(item)

with open('../data/angular_filtered/subsets/generation/test_all.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [3]:

# 确保JSON数据是一个列表
if not isinstance(data, list):
    raise ValueError("JSON数据不是一个列表")

# 随机选择100个项目
sample_size = 50
if len(data) < sample_size:
    raise ValueError("数据量少于50个项目")

initial_sample = random.sample(data, sample_size)

# 从剩余的项目中再随机选择100个项目
remaining_data = [item for item in data if item not in initial_sample]
# second_sample = random.sample(remaining_data, sample_size)

# 将选定的项目写入一个新的JSON文件
with open('../data/angular_filtered/subsets/human_eval/no_human_ref_test.json', 'w', encoding='utf-8') as f:
    json.dump(initial_sample, f, ensure_ascii=False, indent=4)
    
# 将选定的项目写入一个新的JSON文件
# with open('../data/angular_filtered/subsets/human_eval/human_ref_test.json', 'w', encoding='utf-8') as f:
#     json.dump(second_sample, f, ensure_ascii=False, indent=4)

print("随机提取的50个项目已保存到output.json文件中。")


随机提取的50个项目已保存到output.json文件中。


In [4]:
## 统计随机提取的50个项目的类型分布
types = {}
for item in initial_sample:
    if item['type'] not in types:
        types[item['type']] = 0
    types[item['type']] += 1

print("类型分布：")
for key, value in types.items():
    print(f"{key}: {value}")

类型分布：
refactor: 8
docs: 10
ci: 4
test: 5
feat: 9
chore: 5
fix: 7
style: 1
build: 1


In [5]:
import json
import random

# 读取原始数据文件
with open('../data/angular_filtered/subsets/human_eval/no_human_ref_test.json', 'r', encoding='UTF-8') as file:
    data = json.load(file)

mapping = {}

# 遍历每个对象并替换字段名称
for i, item in enumerate(data):
    item_id = f"Object-{i+1}"
    mapping[item_id] = {}
    
    keys = list(item.keys())
    keys_to_rename = [key for key in keys if key not in ['diff', 'type']]
    
    random.shuffle(keys_to_rename)
    
    # 创建一个新的字典来保存重命名后的字段
    new_item = {"diff": item["diff"]}
    
    for idx, key in enumerate(keys_to_rename, 1):
        new_key = str(idx)
        mapping[item_id][new_key] = key
        new_item[new_key] = item[key]
    
    # 替换原有的item
    data[i] = new_item

# 将修改后的数据写入新的JSON文件
with open('../data/angular_filtered/subsets/human_eval/no_human_ref_test_shuffled.json', 'w', encoding='UTF-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

# 将映射关系写入映射文件
with open('../data/angular_filtered/subsets/human_eval/no_human_ref_test_mapping.json', 'w', encoding='UTF-8') as file:
    json.dump(mapping, file, ensure_ascii=False, indent=4)



In [3]:
import json

# 读取output.json文件
with open('../data/angular_filtered/subsets/human_eval/no_human_ref_test_shuffled.json', 'r') as file:
    data = json.load(file)

# 打开markdown文件用于写入
with open('../data/angular_filtered/subsets/human_eval/no_human_ref_test_shuffled.md', 'w') as md_file:
    for i, item in enumerate(data, 1):
        md_file.write(f"## Item {i}\n\n")
        
        if 'diff' in item:
            md_file.write("```diff\n")
            md_file.write(f"{item['diff']}\n")
            md_file.write("```\n\n")
        
        for key, value in item.items():
            if key != 'diff':
                md_file.write(f"{key}: {value}\n\n")
        
        md_file.write("best description number: \n\n")
        md_file.write("---\n\n")

print("处理完成，生成了output.md文件。")


处理完成，生成了output.md文件。


In [1]:
import json

# Read the JSON file
with open('../data/angular_filtered/subsets/human_eval/no_human_ref_test_shuffled.json', 'r', encoding='UTF-8') as file:
    data = json.load(file)

# Define evaluation criteria
evaluation_criteria = """
Instruction: Your task is to evaluate the commit messages based on the commit code diffs, so first look at the code diffs and determine what the code changes are, and you will be given 2 commit messages generated by different models for each commit. **You need to score each commit message from the following criteria from the perspectives of Expressiveness, and Informativeness, with a minimum score of 0 and a maximum score of 4. Finally, compare the two commit messages given for each commit and select the Best description number (1 or 2).**

Evaluation Criteria:
| Expressiveness | Description                              |
| -------------- | ---------------------------------------- |
| 0              | Cannot read and understand.              |
| 1              | The content is difficult to read and understand.          |
| 2              | The content is somewhat readable and understandable. |
| 3              | The content is mostly readable and understandable.   |
| 4              | The content is very easy to read and understand.          |

| Informativeness | Description                                                  |
| --------------- | ------------------------------------------------------------ |
| 0               | No information about the code change is provided.             |
| 1               | Some crucial information is missing, making it hard to understand the code changes. |
| 2               | Some information is missing, but the missing parts are not essential for understanding the code changes. |
| 3               | There is some missing information, but none of it is necessary to understand the code changes. |
| 4               | All necessary information about the code change is included.                                 |
"""
with open('../data/angular_filtered/subsets/human_eval/no_human_ref_test_shuffled.md', 'w', encoding='UTF-8') as md_file:
    # Print evaluation criteria once at the beginning
    md_file.write(evaluation_criteria)
    
    # Parse each item and format the output into markdown
    for i, item in enumerate(data, 1):
        md_file.write(f"## Item {i}\n\n")
        diff = item['diff']
        md_file.write("````diff\n")
        md_file.write(diff)
        md_file.write("````\n\n")
    
        for key, value in item.items():
            if key != 'diff':
                value = value.replace('\n', ', ')
                value = value.replace('\r', ', ')
                md_file.write(f"{key}: '{value}'\n\n")
                # md_file.write("Conciseness: \n")
                md_file.write("Expressiveness: \n")
                md_file.write("Informativeness: \n\n")
        
        md_file.write("**Best description number:** \n\n")
        md_file.write("---\n\n")


In [2]:
def generate_file(filename, num_items=50):
    with open(filename, 'w') as file:
        for i in range(1, num_items + 1):
            file.write(f"item{i}-message1:\n")
            file.write("Expressiveness:\n")
            file.write("Informativeness:\n\n")
            file.write(f"item{i}-message2:\n")
            file.write("Expressiveness:\n")
            file.write("Informativeness:\n\n")
            file.write("Best description number:\n\n")
            file.write("---\n\n")

if __name__ == "__main__":
    generate_file("../data/angular_filtered/subsets/human_eval/no_human_ref_test.txt")

## Analyze the result from human evaluation

In [32]:
import re
import json
# 正则表达式匹配模式
pattern1 = r'Best description number:\s*(\d+)'
pattern2 = r'Best description number:\s*item(\d+)-message(\d+)'

for file_name in range(1, 6):
    # 读取文件内容
    file_path = f'../data/angular_filtered/subsets/human_eval/people/no_human_ref_test_{file_name}.txt'
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    json_data = []
    # 按照item分割文件内容
    items = content.strip().split('---\n')
    best_description_counts = {1: 0, 2: 0}
    for index, item in enumerate(items):
        item = [text for text in item.strip().split('\n') if text]
        ex_1_match = re.match("Expressiveness:\s*(\d+)", item[1])
        if not ex_1_match:
            raise ValueError(f"无法匹配表达性: {item[1]}")
        ex_1 = ex_1_match.group(1)
        in_1_match = re.match("Informativeness:\s*(\d+)", item[2])
        if not in_1_match:
            raise ValueError(f"无法匹配信息性: {item[2]}")
        in_1 = in_1_match.group(1)
        ex_2_match = re.match("Expressiveness:\s*(\d+)", item[4])
        if not ex_2_match:
            raise ValueError(f"无法匹配表达性: {item[4]}")
        ex_2 = ex_2_match.group(1)
        in_2_match = re.match("Informativeness:\s*(\d+)", item[5])
        if not in_2_match:
            raise ValueError(f"无法匹配信息性: {item[5]}")
        in_2 = in_2_match.group(1)
        # use pattern1 and pattern2 to match the best description number
        match1 = re.match(pattern1, item[6])
        match2 = re.match(pattern2, item[6])
        if match1:
            best_id = int(match1.group(1))
        elif match2:
            best_id = int(match2.group(2))
        else:
            raise ValueError(f"无法匹配最佳描述号: {item[6]}")
        data = {"Item_number": int(index+1), "Expressiveness1": int(ex_1), "Informativeness1": int(in_1), "Expressiveness2": int(ex_2), "Informativeness2": int(in_2), "Best_description_number": best_id}
        json_data.append(data)
    
    with open(f'../data/angular_filtered/subsets/human_eval/people/no_human_ref_test_{file_name}.json', 'w', encoding='utf-8') as file:
        json.dump(json_data, file, ensure_ascii=False, indent=4)

ValueError: 无法匹配表达性: Expressiveness:

In [23]:
items = content.strip().split('---\n')

In [25]:
item = [text for text in items[0].strip().split('\n') if text]

In [26]:
item

['item1-message1:',
 'Expressiveness:2',
 'Informativeness:1',
 'item1-message2:',
 'Expressiveness:3',
 'Informativeness:3',
 'Best description number:item1-message2']

In [ ]:
for item in items:
    

In [2]:

for item in items:
    if item.strip():
        # 提取item编号
        item_match = re.search(r'item(\d+)', item)
        if item_match:
            item_number = item_match.group(1)
            messages = re.findall(r'message\d+:.*?\nExpressiveness:(\d+)\nInformativeness: (\d+)', item, re.DOTALL)
            items_data[f'item{item_number}'] = []
            for message in messages:
                expressiveness, informativeness = message
                items_data[f'item{item_number}'].append({
                    'Expressiveness': int(expressiveness),
                    'Informativeness': int(informativeness)
                })
            
            # 提取Best description number
            best_desc_match = re.search(r'Best description number:(\d)', item)
            if best_desc_match:
                best_description_number = int(best_desc_match.group(1))
                best_description_counts[best_description_number] += 1

# 输出结果
print("Items Data:")
for item, messages in items_data.items():
    print(f"{item}:")
    for message in messages:
        print(f"  {message}")

print("\nBest Description Counts:")
for number, count in best_description_counts.items():
    print(f"  {number}: {count}")

Items Data:
item1:
  {'Expressiveness': 3, 'Informativeness': 1}
item2:
item3:
item4:
item5:
item6:
item7:
item8:
item9:
item10:
item11:
item12:
item13:
item14:
item15:
item16:
item17:
item18:
item19:
item20:
item21:
item22:
item23:
item24:
item25:
item26:
item27:
item28:
item29:
item30:
item31:
item32:
item33:
item34:
item35:
item36:
item37:
item38:
item39:
item40:
item41:
item42:
item43:
item44:
item45:
item46:
item47:
item48:
item49:
item50:

Best Description Counts:
  1: 22
  2: 26
